# Location Intelligence: Traffic Analysis
Location intelligence involves analyzing spatial data to gain insights. In this notebook, we'll simulate traffic data and perform geospatial analysis to detect traffic congestion hotspots.



# Import Necessary Libraries


In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from folium.plugins import HeatMap

# Simulate Traffic Data


In [ ]:

# Generate Random Traffic Data
np.random.seed(42)
num_points = 500  # Number of data points

# Simulate coordinates around a central location (e.g., a city center)
latitudes = np.random.uniform(24.65, 24.75, num_points)
longitudes = np.random.uniform(46.65, 46.75, num_points)

# Simulate traffic density (scale 10 to 100)
traffic_density = np.random.uniform(10, 100, num_points)

# Simulate time of day (0 to 23 hours)
timestamps = np.random.randint(0, 24, num_points)

# Create DataFrame
traffic_data = pd.DataFrame({
    'latitude': latitudes,
    'longitude': longitudes,
    'traffic_density': traffic_density,
    'hour_of_day': timestamps
})

# Display first few rows
traffic_data.head()


,latitude,longitude,traffic_density,hour_of_day
0,24.687454,46.719816,26.661964,15
1,24.745071,46.703610,58.771085,1
2,24.723199,46.680953,88.565125,5
3,24.709866,46.731380,75.900240,20
4,24.665602,46.718473,82.590503,11


# Visualize Traffic Data on a Map


In [ ]:
#Create a Folium Map

center_location = [traffic_data['latitude'].mean(), traffic_data['longitude'].mean()]
traffic_map = folium.Map(location=center_location, zoom_start=13)
#Add Traffic Points to the Map

for idx, row in traffic_data.iterrows():
  folium.CircleMarker(location=(row['latitude'], row['longitude']), radius=row['traffic_density'] / 20, # Scale marker size
                      color='red' if row['traffic_density'] > 70 else 'green', fill=True, fill_opacity=0.6 ).add_to(traffic_map)
#Display the Map

traffic_map

#Traffic Hotspot Detection Using DBSCAN


In [ ]:
#Prepare Data for Clustering

coords = traffic_data[['latitude', 'longitude']].values
#Apply DBSCAN Clustering

db = DBSCAN(eps=0.003, min_samples=5).fit(coords)
labels = db.labels_
#Add Cluster Labels to DataFrame

traffic_data['cluster'] = labels
#Visualize Clusters

unique_labels = set(labels)
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
plt.figure(figsize=(10, 8))
for k, col in zip(unique_labels, colors):
  class_member_mask = (labels == k)
  xy = coords[class_member_mask]
  plt.plot(xy[:, 1], xy[:, 0], 'o', markerfacecolor=tuple(col[:3]), markeredgecolor='k', markersize=8)
plt.title('Traffic Hotspots Detected by DBSCAN')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()


#Create a Traffic Density Heatmap


In [ ]:
#Prepare Heatmap Data

heat_data = traffic_data[['latitude', 'longitude', 'traffic_density']].values.tolist()
#Generate Heatmap

heat_map = folium.Map(location=center_location, zoom_start=13)
HeatMap(heat_data, radius=10, max_zoom=13).add_to(heat_map)
#Display Heatmap

heat_map